# Clientes principales (personas físicas) con posible conflicto de interés (Métrica 4_2_08)
| **Escenario**                                               | **Condición evaluada**                                      | **Resultado**    |
| ----------------------------------------------------------- | ----------------------------------------------------------- | ---------------- |
| **No hay clientes principales declarados**                  | Lista `clientesPrincipales.cliente` no existe o viene vacía | ⚪ **NO_APLICA**  |
| **Cliente principal NO es persona física**                  | Campo “otorgante” distinto a “PERSONA FISICA”               | ⚪ **NO_APLICA**  |
| **Faltan datos esenciales**                                 | Sector productivo vacío, nulo o sin clasificar              | ⚪ **SIN_DATO**   |
| **Sector productivo está en sectores de riesgo (T, D o C)** | `sector ∈ sectores_riesgo[funcion_principal_del_ente]`      | 🔴 **NO_CUMPLE** |
| **Sector NO está en sectores de riesgo**                    | Cliente principal persona física con sector fuera de T/D/C  | 🟢 **CUMPLE**    |
| **Registros incompletos**                                   | Declaración de cliente sin sector definido                  | ⚪ **SIN_DATO**   |


In [1]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME
import unicodedata


METRIC_ID = "4_2_08_CLIENTES_PRINCIPALES_PF_SECTOR_RIESGO"


# -------------------------------------------------------------------
# MATRIZ TEMPORAL - substituir cuando tengas catálogo INEGI real
# -------------------------------------------------------------------
MATRIZ_SECTORES_FUNCIONES = {
    "SALUD": ["T", "D"],
    "OBRAS PUBLICAS": ["T", "C"],
    "EDUCACION": ["D"],
    "SEGURIDAD": ["T", "C"],
}


# -------------------------------------------------------------------
# AUXILIARES
# -------------------------------------------------------------------
def limpiar(v):
    if not v:
        return None
    v = str(v).strip().upper()
    v = ''.join(c for c in unicodedata.normalize('NFD', v)
                if unicodedata.category(c) != 'Mn')
    while "  " in v:
        v = v.replace("  ", " ")
    return v


def extraer(doc, path):
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p)
            elif isinstance(actual, list) and len(actual) > 0:
                actual = actual[0].get(p)
            else:
                return None
        return actual
    except:
        return None


# -------------------------------------------------------------------
# EVALUADOR
# -------------------------------------------------------------------
def evaluar_metrica(doc):

    # 1. Ente público
    ente = limpiar(
        extraer(doc, "declaracion.situacionPatrimonial.datosEmpleoCargoComision.nombreEntePublico")
    )

    if not ente:
        return "SIN_DATO"

    funcion_principal = ente  # se sustituirá con catálogo real
    sectores_riesgo = MATRIZ_SECTORES_FUNCIONES.get(funcion_principal, [])

    # 2. Clientes principales
    clientes = extraer(doc, "declaracion.intereses.clientesPrincipales.cliente")

    if not clientes:
        return "NO_APLICA"

    if not isinstance(clientes, list):
        clientes = [clientes]

    hubo_datos = False

    for cli in clientes:

        otorgante = limpiar(cli.get("otorgante"))
        sector = limpiar(cli.get("sector"))

        # Solo evaluar PERSONAS FÍSICAS
        if otorgante != "PERSONA FISICA":
            continue

        # Falta sector
        if not sector:
            return "SIN_DATO"

        hubo_datos = True

        # Sector de riesgo
        if sector in sectores_riesgo:
            return "NO_CUMPLE"

    if not hubo_datos:
        return "NO_APLICA"

    return "CUMPLE"


# -------------------------------------------------------------------
# MOTOR MONGO
# -------------------------------------------------------------------
def procesar_metrica_4_2_08():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0, "NO_APLICA": 0}
    operaciones = []
    total = 0

    try:
        print(f"\n🚀 Procesando métrica {METRIC_ID}...\n")

        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]

        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        cursor = src.find({}, {
            "_id": 1,
            "declaracion.situacionPatrimonial.datosEmpleoCargoComision": 1,
            "declaracion.intereses.clientesPrincipales": 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1

            try:
                resultado = evaluar_metrica(doc)
            except Exception as e:
                print(f"⚠️ Error documento {doc.get('_id')}: {e}")
                resultado = "SIN_DATO"

            resultados[resultado] += 1

            operaciones.append(
                UpdateOne(
                    {"_id": doc["_id"]},
                    {"$set": {METRIC_ID: resultado}},
                    upsert=True
                )
            )

            if len(operaciones) >= 2000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f" > {total} documentos procesados...")

        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print(f"Total procesados: {total}")
        for k, v in resultados.items():
            print(f"  {k}: {v}")

    except:
        traceback.print_exc()

    finally:
        try:
            client.close()
        except:
            pass
        print("Conexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_4_2_08()



🚀 Procesando métrica 4_2_08_CLIENTES_PRINCIPALES_PF_SECTOR_RIESGO...



c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py:1954: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


 > 2000 documentos procesados...
 > 4000 documentos procesados...
 > 6000 documentos procesados...
 > 8000 documentos procesados...
 > 10000 documentos procesados...
 > 12000 documentos procesados...
 > 14000 documentos procesados...
 > 16000 documentos procesados...
 > 18000 documentos procesados...
 > 20000 documentos procesados...
 > 22000 documentos procesados...
 > 24000 documentos procesados...
 > 26000 documentos procesados...
 > 28000 documentos procesados...
 > 30000 documentos procesados...
 > 32000 documentos procesados...
 > 34000 documentos procesados...
 > 36000 documentos procesados...
 > 38000 documentos procesados...
 > 40000 documentos procesados...
 > 42000 documentos procesados...
 > 44000 documentos procesados...
 > 46000 documentos procesados...
 > 48000 documentos procesados...
 > 50000 documentos procesados...
 > 52000 documentos procesados...
 > 54000 documentos procesados...
 > 56000 documentos procesados...
 > 58000 documentos procesados...
 > 60000 documento

Traceback (most recent call last):
  File "C:\Users\Mauro\AppData\Local\Temp\ipykernel_21384\2094506460.py", line 126, in procesar_metrica_4_2_08
    for doc in cursor:
               ^^^^^^
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\cursor.py", line 1289, in __next__
    return self.next()
           ~~~~~~~~~^^
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\cursor.py", line 1265, in next
    if len(self._data) or self._refresh():
                          ~~~~~~~~~~~~~^^
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\cursor.py", line 1236, in _refresh
    self._send_message(g)
    ~~~~~~~~~~~~~~~~~~^^^
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\cursor.py", line 1108, in _send_message
    response = client._run_operation(
        operation, self._unpack_response, address=self._address
    )
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\_csot.py", line 125, in csot_wrapp